## 대신증권 API cybos plus 사용하기

### 필요 모듈 import

In [1]:
import win32com.client
import pandas as pd
import time
import datetime

### cybos plus 연결 체크

In [2]:
 # 연결 여부 체크
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect
if (bConnect == 0):
    print("PLUS가 정상적으로 연결되지 않음. ")
    exit()
else:
    print("PLUS가 정상적 연결됨")


PLUS가 정상적 연결됨


#### KOSPI 31에 해당하는 종목코드 찾기

In [3]:
kospi31_list = pd.read_csv("./data/kospi31_list.csv")
kospi31_list

,code,name
0,U182,kospi50_index
1,A030200,KT
2,A033780,KT&G
3,A003550,LG
4,A051900,LG생활건강
5,A032640,LG유플러스
6,A066570,LG전자
7,A051910,LG화학
8,A035420,NAVER
9,A005490,POSCO


In [6]:
def get_kospi50_stock_data(stockitems, start, end):
    instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
    nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    
    # 날짜, 종목명, 시가, 고가, 저가, 종가
    column_dailychart = ['code', 'name', 'date', 'open', 'high', 'low', 'close',]
    
    stockitems_dailychart = pd.DataFrame()
    
    s_year, e_year = start[:4], end[:4]
    s_day, e_day = start[4:], end[4:]
    
    instStockChart.SetInputValue(1, ord('1'))
    instStockChart.SetInputValue(5, (0, 2, 3, 4, 5))
    instStockChart.SetInputValue(6, ord('D'))
    instStockChart.SetInputValue(9, ord('1'))

    for idx, stockitem in stockitems.iterrows():
        dailychart = pd.DataFrame()
        row = list(range(len(column_dailychart)))
        rows = list()
        
        instStockChart.SetInputValue(0, stockitem['code'])
        print(stockitem['name'], "주식 데이터를 가져옵니다.")
        
        y = int(s_year)
        while y <= int(e_year):
            # 시작일, 종료일 계산
            if s_year == y:
                s_y, s_d = s_year, s_day
            else:
                s_y, s_d = str(y), '0101'
                
            if e_year == y:
                e_y, e_d = e_year, e_day
            else:
                e_y, e_d = str(y), '1231'
                
            print(s_y + s_d + '~' + e_y + e_d + '의 데이터를 가져옵니다.')
            instStockChart.SetInputValue(3, s_y + s_d)
            instStockChart.SetInputValue(2, e_y + e_d)
            
            # 남은요청 있는지 확인
            remain_request_count = nCpCybos.GetLimitRemainCount(1)
            print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)

            if remain_request_count == 0:
                print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')
                while True:
                    time.sleep(2)
                    remain_request_count = nCpCybos.GetLimitRemainCount(1)

                    if remain_request_count > 0:
                        print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                        break
                    print('대기 중...')
                    
            # BlockRequest
            instStockChart.BlockRequest()

            # GetHeaderValue
            numData = instStockChart.GetHeaderValue(3)
            numField = instStockChart.GetHeaderValue(1)
            # GetDataValue
            print(numData)
            for i in range(numData):
                row[0] = stockitem['code']
                row[1] = stockitem['name']
                row[2] = instStockChart.GetDataValue(0, i)# 날짜
                row[3] = instStockChart.GetDataValue(1, i) # 시가
                row[4] = instStockChart.GetDataValue(2, i) # 고가
                row[5] = instStockChart.GetDataValue(3, i) # 저가
                row[6] = instStockChart.GetDataValue(4, i) # 종가
                rows.append(list(row))
            df =  pd.DataFrame(data = rows, columns= column_dailychart)
            dailychart = pd.concat([dailychart, df])
            y += 1    
        dailychart = dailychart.sort_values("date")
        dailychart = dailychart.drop_duplicates(['date'])
        dailychart = dailychart.reset_index(drop=True)
        stockitems_dailychart = pd.concat([stockitems_dailychart, dailychart])
    return stockitems_dailychart

In [7]:
market = kospi31_list[kospi31_list.code == "U182"]
kospi50index_data = get_kospi50_stock_data(market, "20030101", "20210914")
kospi50index_data

kospi50_index 주식 데이터를 가져옵니다.
20030101~20031231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  60
247
20040101~20041231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  59
249
20050101~20051231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  58
249
20060101~20061231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  57
247
20070101~20071231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  56
246
20080101~20081231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  55
248
20090101~20091231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  54
253
20100101~20101231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  53
251
20110101~20111231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  52
248
20120101~20121231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  51
248
20130101~20131231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  50
247
20140101~20141231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  49
245
20150101~20151231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  48
248
20160101~20161231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  47
246
20170101~20171231의 데이터를 가져옵니다.
U182 kospi50_ind

,code,name,date,open,high,low,close
0,U182,kospi50_index,20030102,594.500000,597.070007,584.559998,594.840027
1,U182,kospi50_index,20030103,611.070007,622.500000,608.710022,621.150024
2,U182,kospi50_index,20030106,624.799988,628.260010,618.059998,626.940002
3,U182,kospi50_index,20030107,639.080017,639.419983,612.830017,613.109985
4,U182,kospi50_index,20030108,617.380005,621.359985,611.400024,612.520020
...,...,...,...,...,...,...,...
4625,U182,kospi50_index,20210909,2846.560059,2858.580078,2810.719971,2815.219971
4626,U182,kospi50_index,20210910,2818.320068,2830.219971,2805.340088,2821.600098
4627,U182,kospi50_index,20210913,2810.709961,2834.520020,2804.149902,2825.580078
4628,U182,kospi50_index,20210914,2832.570068,2859.149902,2831.300049,2842.000000


In [8]:
stocks = kospi31_list[kospi31_list.code != "U182"]
kospi31_data = get_kospi50_stock_data(stocks, "20030101", "20210914")
kospi31_data

KT 주식 데이터를 가져옵니다.
20030101~20031231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  41
247
20040101~20041231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  40
249
20050101~20051231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  39
249
20060101~20061231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  38
247
20070101~20071231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  37
246
20080101~20081231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  36
248
20090101~20091231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  35
253
20100101~20101231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  34
251
20110101~20111231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  33
248
20120101~20121231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  32
248
20130101~20131231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  31
247
20140101~20141231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  30
245
20150101~20151231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  29
248
20160101~20161231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  28
246
20170101~20171231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  27
243
20180101~20181231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  26
244
20190101~20191231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  25


248
20210101~20211231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  29
177
NAVER 주식 데이터를 가져옵니다.
20030101~20031231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  28
247
20040101~20041231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  27
249
20050101~20051231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  26
249
20060101~20061231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  25
247
20070101~20071231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  24
246
20080101~20081231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  23
248
20090101~20091231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  22
253
20100101~20101231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  21
251
20110101~20111231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  20
248
20120101~20121231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  19
248
20130101~20131231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  18
247
20140101~20141231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  17
245
20150101~20151231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  16
248
20160101~20161231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  15
246
20170101~20171231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  14
243


20180101~20181231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  19
244
20190101~20191231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  18
246
20200101~20201231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  17
248
20210101~20211231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  16
177
기아 주식 데이터를 가져옵니다.
20030101~20031231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  15
247
20040101~20041231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  14
249
20050101~20051231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  13
249
20060101~20061231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  12
247
20070101~20071231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  11
246
20080101~20081231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  10
248
20090101~20091231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  9
253
20100101~20101231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  8
251
20110101~20111231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  7
248
20120101~20121231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  6
248
20130101~20131231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  5
247
20140101~20141231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  4
245
20150101~20151231의 데이터를 가져옵니다.
A000270 기아 남은 요청 :  3

20200101~20201231의 데이터를 가져옵니다.
A000810 삼성화재 남은 요청 :  4
248
20210101~20211231의 데이터를 가져옵니다.
A000810 삼성화재 남은 요청 :  3
177
신한지주 주식 데이터를 가져옵니다.
20030101~20031231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  2
247
20040101~20041231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  1
249
20050101~20051231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
249
20060101~20061231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  59
247
20070101~20071231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  58
246
20080101~20081231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  57
248
20090101~20091231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  56
253
20100101~20101231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  55
251
20110101~20111231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  54
248
20120101~20121231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  53
248
20130101~20131231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  52
247
20140101~20141231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 :  51
245
20150101~20151231의 데이터를 가져옵니다.
A055550 신한지주 남은 요청 : 

A009540 한국조선해양 남은 요청 :  52
246
20200101~20201231의 데이터를 가져옵니다.
A009540 한국조선해양 남은 요청 :  51
248
20210101~20211231의 데이터를 가져옵니다.
A009540 한국조선해양 남은 요청 :  50
177
한온시스템 주식 데이터를 가져옵니다.
20030101~20031231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  49
247
20040101~20041231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  48
249
20050101~20051231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  47
249
20060101~20061231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  46
247
20070101~20071231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  45
246
20080101~20081231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  44
248
20090101~20091231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  43
253
20100101~20101231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  42
251
20110101~20111231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  41
248
20120101~20121231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  40
248
20130101~20131231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  39
247
20140101~20141231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  38
245
20150101~20151231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  37
248
20160101~20161231의 데이터를 가져옵니다.


,code,name,date,open,high,low,close
0,A030200,KT,20030102,50800,51000,49600,49900
1,A030200,KT,20030103,50600,52400,50200,52400
2,A030200,KT,20030106,51900,52700,51400,52400
3,A030200,KT,20030107,53400,53400,51300,51500
4,A030200,KT,20030108,52200,53000,51600,52000
...,...,...,...,...,...,...,...
4625,A005380,현대차,20210909,212500,212500,208000,208500
4626,A005380,현대차,20210910,208500,209500,204000,206500
4627,A005380,현대차,20210913,206500,206500,204000,205000
4628,A005380,현대차,20210914,206500,211500,206000,209000


#### KOSPI index와 개별 주식 데이터 합치고 저장하기

In [9]:
kospi31_index_stocks = pd.concat([kospi50index_data, kospi31_data])
kospi31_index_stocks

,code,name,date,open,high,low,close
0,U182,kospi50_index,20030102,594.500000,597.070007,584.559998,594.840027
1,U182,kospi50_index,20030103,611.070007,622.500000,608.710022,621.150024
2,U182,kospi50_index,20030106,624.799988,628.260010,618.059998,626.940002
3,U182,kospi50_index,20030107,639.080017,639.419983,612.830017,613.109985
4,U182,kospi50_index,20030108,617.380005,621.359985,611.400024,612.520020
...,...,...,...,...,...,...,...
4625,A005380,현대차,20210909,212500.000000,212500.000000,208000.000000,208500.000000
4626,A005380,현대차,20210910,208500.000000,209500.000000,204000.000000,206500.000000
4627,A005380,현대차,20210913,206500.000000,206500.000000,204000.000000,205000.000000
4628,A005380,현대차,20210914,206500.000000,211500.000000,206000.000000,209000.000000


In [10]:
kospi31_index_stocks.to_csv("./data/kospi31-2003-2021.csv", index=False)